In [2]:
from tqdm import tqdm

from typing import Dict, List, Tuple
ArticleName = str
Text = str
Term = str
CollectionData = None
RankingParams = None

### 1. Датасет
В качестве документов для поиска, требуется использовать статьи википедии, скачанные в одном из предыдущих домашних заданий.
Реализуйте получение содержимого документа по названию статьи в соответствии с выбранной вами схемы хранения документов. Также, преобразуйте содержимое в текст (не html) любым разумным способом. Постарайтесь при этом вырезать заведомо бесполезные для поиска данные со страницы.

In [3]:
import parser1

for article_name in ["Software_Development_Kit", "Gangrene", "COVID-19_pandemic_in_Belarus", "Guitar_Hero:_Aerosmith"]:
    print(f'Article: {article_name}\n{parser1.get_article_text(article_name)}\n\n')

Article: Software_Development_Kit
A software development kit (SDK or "devkit") is usually a set of development tools that allows a software developer to create applications for a certain software package, software framework, hardware platform, computer system, video game console, operating system, or similar platform. SDKs vary greatly between a simple application programming interface to hardware used to simulate a system. Computers


Article: Gangrene
Gangrene is a serious medical condition that causes the decay and death of body tissue, usually in the extremities such as the fingers, hands, toes, and feet. The two main types of gangrene are dry gangrene and wet gangrene. A third less common type is a form of wet gangrene known as gas gangrene. A very rare type which affects the internal organs is known as internal gangrene. Dry gangrene is usually caused by a loss of blood supply to the affected area, such as may happen following an injury which damages the blood vessels to the affe

D:\parser1.py:28: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 28 of the file D:\parser1.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


Чтобы процесс занимал разумное время, поиск требуется производить только по некоторому заданному набору документов. Список целевых статей описан в файле selected_docs.tsv по одной на строку.

In [4]:
from multiprocessing import Pool

def load_docs(selected_docs_fn: ArticleName, threads: int = 4) -> Dict[ArticleName, Text]:    
    docs = {}
    pool = Pool(threads)
    tasks = []
    for line in tqdm(open(selected_docs_fn, encoding='utf8')):
        article_name = line.strip()
        tasks.append((article_name, pool.apply_async(parser1.get_article_text, (article_name, ))))
    for article_name, task in tqdm(tasks):
        docs[article_name] = task.get(10**6)
    return docs
    
docs = load_docs("./selected_docs.tsv", 32)
print(f'{len(docs)} docs loaded')

15229it [00:00, 31464.88it/s]
100%|███████████████████████████████████████████████████████████████████████████| 15229/15229 [02:04<00:00, 122.40it/s]

15229 docs loaded


Проверим, сколько документов из списка неправильно скачались (так как не было ретраев и проверки статус кода) и отфильтруем нормальные для дальнейшей работы

In [20]:
fdocs = {k: v for k, v in docs.items() if len(v) > 0}
print(len(docs) - len(fdocs))
fdocs, docs = docs, fdocs

830


### 2. Поиск
Реализуйте разбиение текста на термы.

In [21]:
import string
import nltk

from collections import defaultdict 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

def make_terms(text: Text) -> List[Term]:
    return [porter.stem(word) for word 
            in text.lower().translate(str.maketrans('', '', string.punctuation)).split()
            if word not in stop_words]

for text in ["Hello, world!", 
             "Guitar Hero: Aerosmith", 
             "So Far, So Good... So What!", 
             "such as mantı, döner, kebabs, Turkish delight, baklava, börek, köfte, and other foods",
            ]:
    print(f'{text} -> {make_terms(text)}')
    
documents_terms = {k: make_terms(v) for k, v in docs.items()}
    
print(documents_terms['A'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pavel.tyletsky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Hello, world! -> ['hello', 'world']
Guitar Hero: Aerosmith -> ['guitar', 'hero', 'aerosmith']
So Far, So Good... So What! -> ['far', 'good']
such as mantı, döner, kebabs, Turkish delight, baklava, börek, köfte, and other foods -> ['mantı', 'döner', 'kebab', 'turkish', 'delight', 'baklava', 'börek', 'köfte', 'food']
['first', 'letter', 'english', 'alphabet', 'small', 'letter', 'α', 'use', 'lower', 'case', 'vowel', 'spoken', 'ā', 'said', 'long', 'diphthong', 'ĕ', 'similar', 'alpha', 'greek', 'alphabet', 'surpris', 'stand', 'sound', 'alpha', 'omega', 'last', 'letter', 'greek', 'alphabet', 'mean', 'begin', 'end', 'music', 'notat', 'letter', 'symbol', 'note', 'scale', 'b', 'g', 'binari', 'number', 'letter', '01000001', 'letter', 'use', 'repres', 'team', 'old', 'tv', 'show', 'ateam', 'capit', 'written', 'use', 'capit', 'start', 'sentenc', 'write', 'letter', 'phoenician', 'alphabet', 'aleph', 'symbol', 'came', 'simpl', 'pictur', 'ox', 'head', 'phoenician', 'letter', 'help', 'make', 'basic', '

Рассчитайте статистики термов и документов, которые понадобятся для реализации моделей tfidf и BM25: частота терма для документа, обратная документная частота терма и прочие.

In [22]:
term_index = defaultdict(lambda: defaultdict(int))

for name, content in documents_terms.items():
    for term in content:
        term_index[term][name] += 1
        
print(term_index['letter'])

defaultdict(<class 'int'>, {'Warsaw_Uprising': 1, 'Allele': 2, 'Orlande_de_Lassus': 2, 'God': 3, 'Bible': 1, 'Jawa_Motors': 1, 'Croatian_language': 6, 'Edward_Drinker_Cope': 4, 'English_Civil_War': 1, 'Santa_Claus_lore': 1, 'Chess': 3, 'Unknown_(math)': 4, 'Function_(mathematics)': 2, 'Wisconsin_in_the_American_Civil_War': 1, 'Four_color_theorem': 2, 'John_Campbell_Ross': 1, 'Latin_alphabet_in_Turkish_countries': 10, 'Imaginary_number': 1, 'Calculus': 2, 'Joseph_Whittaker_(botanist)': 1, 'United_States_Army_trucks': 1, 'Levenshtein_distance': 1, 'Intergovernmental_Panel_on_Climate_Change': 2, 'List_of_dialling_codes_in_the_United_Kingdom': 1, 'Manuela_Saenz': 2, 'List_of_counties_in_Michigan': 1, 'Atomic_nucleus': 2, 'One_Day_in_the_Life_of_Ivan_Denisovich': 1, 'Edgar_Cayce': 2, 'A_Tale_of_Two_Cities': 3, 'Breadboard': 1, 'List_of_Latin_phrases_(I)': 1, 'Johann_Wolfgang_von_Goethe': 3, 'Jesus': 8, 'Shake_It_Up_(TV_series)': 2, 'Anagram': 1, 'Article_One_of_the_United_States_Constitutio

In [23]:
from math import log

def tf(index, documents):
    tfs = defaultdict(float)
    for key, value in index.items():
        for document, count in value.items():
            tfs[(key, document)] = count * 1. / len(documents[document])
            
    return tfs

def idf(index, documents):
    return {term: log(len(documents) * 1. / len(value)) for term, value in index.items()}

idfs = idf(term_index, documents_terms)
tfs = tf(term_index, documents_terms)

def tf_idf(tf, idf):
    return {k: tf_value * idf[k[0]] for k, tf_value in tf.items()}

tf_idf = tf_idf(tfs, idfs)

Реализуйте поиск лучших 10 документов в модели tfidf и BM25 с параметрами b = 1, k1 = 1, k2=1.
Строить инвертированный индекс не требуется (но и не запрещается).

In [24]:
def test_search(search):
    for query in ["coronovirus in belarus",
              "who won junior eurovision in 2005",
              "science about full-text search",
             ]:
        result = search(query)[:5]
        print(f"[{query}]")
        for article_name, score in result:
            print(f"{score:7.2f}  {article_name}")
        print("\n")

In [25]:
from itertools import chain, combinations

def all_subsets(ss):
    return chain(*map(lambda x: combinations(ss, x), range(len(ss),0, -1)))

def search_tfidf(query):
    query = make_terms(query)
    subqueries = all_subsets(query)
    
    result_subsets = defaultdict(int)
    for subset in subqueries:
        sets = [set(term_index[term].keys()) for term in subset]

        result = sets[0] or set()
        for i in range(1, len(sets)):
            result = result & sets[i]
        
        temporary_range = []
        for document in result:
            temporary_range.append((document, sum(map(lambda term: tf_idf.get((term, document)) or 0, subset))))
        
        for doc, score in temporary_range:
            result_subsets[doc] += score

    return list(sorted(result_subsets.items(), key=lambda x: -x[1]))

test_search(search_tfidf)

[coronovirus in belarus]
   0.51  Time_in_Belarus
   0.29  COVID-19_pandemic_in_Belarus
   0.27  Daugava_River
   0.26  Bug_River
   0.18  Eurasian_Union


[who won junior eurovision in 2005]
   0.80  Junior_Eurovision_Song_Contest_2019
   0.66  Herbie:_Fully_Loaded
   0.52  Katherine_Hansen
   0.46  Junior_Eurovision_Song_Contest_2015
   0.42  Junior_Eurovision_Song_Contest_2004


[science about full-text search]
   0.87  Information_retrieval
   0.55  Systems_science
   0.50  Aliweb
   0.49  Natural_sciences
   0.48  Molecular_gastronomy




In [26]:
from statistics import mean 

def search_bm25(query, b, k1, k2, type):
    query_terms = defaultdict(int)
    query = make_terms(query)

    for term in query:
        query_terms[term] += 1
    
    avg_doc_len = mean([len(doc) for doc in documents_terms.values()])

    documents_range = []
    for name, content in documents_terms.items():
        if len(content) == 0:
            continue
        K = k1 * ((1 - b) + b * (len(content) * 1. / avg_doc_len))
        result = sum(
                (idfs.get(term) or 0) * (k1 + 1) * term_index[term][name] * 1. / (K + term_index[term][name]) * \
                (k2 + 1) * query_terms[term] * 1. / (k2 + query_terms[term]) \
            for term in query)
        documents_range.append((name, result))
        
    return sorted(documents_range, key=lambda x: -x[1])

ranking_params = {'type': 'bm25', "b" : 1, "k1" : 1, "k2" : 1} 
test_search(lambda x: search_bm25(x, **ranking_params))

[coronovirus in belarus]
  10.13  Time_in_Belarus
   9.80  COVID-19_pandemic_in_Belarus
   9.75  Daugava_River
   9.72  Bug_River
   9.38  Eurasian_Union


[who won junior eurovision in 2005]
  20.65  Junior_Eurovision_Song_Contest_2019
  19.01  Junior_Eurovision_Song_Contest_2015
  18.61  Junior_Eurovision_Song_Contest_2004
  18.54  Junior_Eurovision_Song_Contest_2014
  15.16  Blue_(group)


[science about full-text search]
  23.33  Information_retrieval
  12.48  On-Line_Encyclopedia_of_Integer_Sequences
  11.79  Scientist
  10.98  Citizen_science
  10.69  Binary_search




### 3. Оптимизация качества
Для измерения качества поиска вам предоставляется список из пар (запрос, название статьи), которая означает, что по данному запросу данная статья является релевантной (а остальные – нерелевантны). Пары описаны в файле queries.tsv по одной на строку.

In [34]:
def load_queries(queries_fn: ArticleName) -> List[Tuple[Text, ArticleName]]:
    queries = []
    for line in open(queries_fn):
        query, answer = line.rstrip().split('\t', 1)
        queries.append((query, answer))
    return queries

queries = list(filter(lambda x: x[1] in docs, load_queries("./queries.tsv")))
for query, answer in queries:
    assert answer in docs
    
print(f'{len(queries)} queries loaded')
for query, article_name in queries[:5]:
    print(f'{query} -> {article_name}')

199 queries loaded
animals that have shells and live in water -> Shell_(zoology)
how many different types of scorpions are there -> Scorpion
describe the structure of a scientific name for a species -> Binomial_nomenclature
what are the 3 types of plastids in plant cells -> Plastid
who named the cell and how did he come up with that name -> Cell_theory


Оценим поиск по нескольким метрикам: accuracy – доля запросов, где на первой позиции был найден релевантный документ; accuracy@10 – доля запросов, где релевантный документ попал в первую десятку, mrr@10 – средняя обратная позиция релевантного документа в первой десятке.

In [35]:
def search(query, ranking_params = None):
    if ranking_params["type"] == "bm25":
        return search_bm25(query, **ranking_params)
    elif ranking_params["type"] == "tfidf":
        return search_tfidf(query)

def run(
    title,
    queries: List[Tuple[Text, ArticleName]],
    collection_data: CollectionData,
    ranking_params: RankingParams) -> None:
    accuracy = 0.0
    accuracy10 = 0.0
    rr = 0.0
    processed = 0
    with tqdm(queries) as progress:
        for query, answer in progress:
            result = search(query, ranking_params)[:10]
            
            rank = None
            for position, (article_name, score) in enumerate(result):
                if article_name == answer:
                    rank = position + 1
                    break
                
            if rank is not None:
                accuracy += (rank == 1)
                accuracy10 += (rank <= 10)
                rr += 1.0 / rank
                
            processed += 1
            progress.set_description(f'Acc: {accuracy/processed:0.2f}, Acc10: {accuracy10/processed:0.2f}, RR: {rr/processed:0.2f}')
    print(f'{title}\n  Accuracy: {accuracy/processed:0.2f}\n  Accuracy10: {accuracy10/processed:0.2f}\n  RR: {rr/processed:0.2f}')

ranking_params = {"type": "tfidf"} 
run("TFIDF", queries, documents_terms, ranking_params)

ranking_params = {"type": "bm25", "b" : 1, "k1" : 1, "k2" : 1} 
run("BM25", queries, documents_terms, ranking_params)

Acc: 0.00, Acc10: 0.50, RR: 0.07:   1%|▍                                               | 2/199 [00:00<00:16, 12.20it/s]

TFIDF
  Accuracy: 0.17
  Accuracy10: 0.45
  RR: 0.25


Acc: 0.18, Acc10: 0.48, RR: 0.26: 100%|██████████████████████████████████████████████| 199/199 [00:16<00:00, 12.39it/s]

BM25
  Accuracy: 0.18
  Accuracy10: 0.48
  RR: 0.26


Подберите оптимальные параметры BM25 для этого набора запросов и документов.

In [36]:
import numpy as np

for b in np.linspace(0, 1, 9):
    for k1 in range(1, 5):
        for k2 in range(1, 5):
            ranking_params = { "type": "bm25", "b": b, "k1": k1, "k2": k2}
            run("BM25" + str(ranking_params), queries, documents_terms, ranking_params)

Acc: 0.00, Acc10: 0.00, RR: 0.00:   1%|▏                                               | 1/199 [00:00<00:29,  6.62it/s]

BM25{'type': 'bm25', 'b': 0.0, 'k1': 1, 'k2': 1}
  Accuracy: 0.14
  Accuracy10: 0.33
  RR: 0.20


Acc: 0.00, Acc10: 0.00, RR: 0.00:   1%|▏                                               | 1/199 [00:00<00:31,  6.21it/s]

BM25{'type': 'bm25', 'b': 0.0, 'k1': 1, 'k2': 2}
  Accuracy: 0.14
  Accuracy10: 0.33
  RR: 0.20


Acc: 0.00, Acc10: 0.00, RR: 0.00:   1%|▏                                               | 1/199 [00:00<00:31,  6.25it/s]

...

BM25{'type': 'bm25', 'b': 1.0, 'k1': 4, 'k2': 2}
  Accuracy: 0.23
  Accuracy10: 0.49
  RR: 0.31


Acc: 0.00, Acc10: 0.00, RR: 0.00:   1%|▏                                               | 1/199 [00:00<00:29,  6.67it/s]

BM25{'type': 'bm25', 'b': 1.0, 'k1': 4, 'k2': 3}
  Accuracy: 0.23
  Accuracy10: 0.49
  RR: 0.31


Acc: 0.23, Acc10: 0.49, RR: 0.31: 100%|██████████████████████████████████████████████| 199/199 [00:34<00:00,  5.72it/s]

BM25{'type': 'bm25', 'b': 1.0, 'k1': 4, 'k2': 4}
  Accuracy: 0.23
  Accuracy10: 0.49
  RR: 0.31


BM25{'type': 'bm25', 'b': 0.625, 'k1': 4, 'k2': 1}
  Accuracy: 0.28
  Accuracy10: 0.57
  RR: 0.36